# Homework 5
For a description of the HousePrices.csv dataset, visit:</br>
https://gitlab.com/CEADS/DrKerby/python/blob/master/HousePrices_description.txt

In [2]:
# Just execute this cell
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

fulldf = pd.read_csv('https://gitlab.com/CEADS/DrKerby/python/raw/master/HousePrices.csv')
fulldf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


**Question 1** (10 points)<br/>
Explore the dataset using three different methods 

In [3]:
# Sums the number of null values in each column
fulldf.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

In [4]:
# Finds the count, mean, std, min, Q1, Q2, Q3, and the max of the data set
fulldf.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
# Determines the dimensions
fulldf.shape

(1460, 81)

In [7]:
# Shows the unique neighborhoods in the dataset
fulldf['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

There are 81 columns in our dataset. One column is the ID (which is not valuable data but an after-label), and another column is the Sale Price, which is what we will try to predict with machine learning. Therefore, we have 79 possible factors to use to create our model. Of these, we will keep the following 16:<br/>
*MSZoning, LotArea, Neighborhood, BldgType, OverallQual, OverallCond, YearBuilt, TotalBsmtSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, GarageCars, SaleCondition*

**Question 2** (10 points)<br/>
Create a DataFrame `housedf` which keeps only these 16 columns from `fulldf`. Print `house.head()` and `house.shape` to show you did it right.

In [8]:
# Creates a dataframe with 16 columns of the orginal data frame
housedf = fulldf[['MSZoning', 'LotArea', 'Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'YearBuilt', 'TotalBsmtSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'GarageCars', 'SaleCondition']]
housedf.head()

,MSZoning,LotArea,Neighborhood,BldgType,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,SaleCondition
0,RL,8450,CollgCr,1Fam,7,5,2003,856,1710,1,0,2,1,3,2,Normal
1,RL,9600,Veenker,1Fam,6,8,1976,1262,1262,0,1,2,0,3,2,Normal
2,RL,11250,CollgCr,1Fam,7,5,2001,920,1786,1,0,2,1,3,2,Normal
3,RL,9550,Crawfor,1Fam,7,5,1915,756,1717,1,0,1,0,3,3,Abnorml
4,RL,14260,NoRidge,1Fam,8,5,2000,1145,2198,1,0,2,1,4,3,Normal


In [9]:
housedf.shape

(1460, 16)

**Question 3** (10 points) <br/>
Place the `SalePrice` column (from `fulldf`) into a variable called `answers`. Show `answers.head()`.

In [10]:
# Creates a dataframe with the SalePrice column
answers = fulldf[['SalePrice']]
answers.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


**Question 4** (10 points) <br/>
12 of our factors in `housedf` are numeric. 4 are categorical. The 4 categorical factors (MSZoning, Neighborhood, BldgType, and SaleCondition) need to be converted using `pandas` `get_dummies`. Do this and assign the result into a new DataFrame called `df`. Print `df.head()` to make sure you completed it correctly. `df` should now have 53 columns. In a separate cell, print `df.columns` so you can see how `get_dummies` handled the categorical factors, and what our 53 factors now are.

*Note:* OverallQual and OverallCond are ordinal. They are rankings from 1 to 10 on the overall quality and condition of the home, respectively. As a ranking of 10 (very excellent) should contribute more to the value of a home than a ranking of 1 (very poor), it is an acceptable approximation to keep these as numeric factors rather than use `get_dummies` and add another 18 factors to our model. However, this means the relationship within OverallQual and OverallCond will be forced to be linear among the rankings and may not represent its real impact on value. You are welcome to also analyze converting these factors using `get_dummies` if you are curious about it, but you do not have to.

In [11]:
# Converts the 4 categorical factors into numerical values
df = pd.get_dummies(housedf, columns=['MSZoning', 'Neighborhood', 'BldgType', 'SaleCondition'])
df.head()

,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,8450,7,5,2003,856,1710,1,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,9600,6,8,1976,1262,1262,0,1,2,0,3,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
2,11250,7,5,2001,920,1786,1,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,9550,7,5,1915,756,1717,1,0,1,0,3,3,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,14260,8,5,2000,1145,2198,1,0,2,1,4,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [12]:
print(df.shape)
df.columns

(1460, 53)


Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'TotalBsmtSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'GarageCars', 'MSZoning_C (all)', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Neighborhood_Blmngtn',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker',
       'BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs

**Question 5** (10 points) <br/>
Now split your `df` and `answers` dataset into a training and testing set, using `train_test_split`. Call the results `Xtrain`, `Xtest`, `ytrain`, and `ytest`. Print `head()` and `shape` for each to ensure the data was split correctly.

In [0]:
#Split dataset into a training and testing set
Xtrain, Xtest, ytrain, ytest = train_test_split(df, answers['SalePrice'], test_size=1/5)

In [14]:
print(Xtrain.shape)
Xtrain.head()

(1168, 53)


,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
432,1920,5,5,1971,765,1365,1,0,1,1,2,2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
413,8960,5,6,1927,1008,1028,0,0,1,0,2,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1037,9240,8,5,2001,1055,2263,0,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
258,12435,7,5,2001,963,1792,0,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1201,10400,7,5,1998,866,1779,0,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [15]:
print(Xtest.shape)
Xtest.head()

(292, 53)


,LotArea,OverallQual,OverallCond,YearBuilt,TotalBsmtSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,GarageCars,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
620,8248,3,3,1914,864,864,1,0,1,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
305,10386,8,5,2004,2000,2000,1,0,2,0,3,3,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
849,9600,6,7,1976,528,1855,0,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
962,2308,6,6,1976,804,1548,1,0,2,1,3,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
940,12640,6,5,1976,1728,1728,0,0,2,0,4,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


In [16]:
print(ytrain.shape)
ytrain.head()

(1168,)


432     122500
413     115000
1037    287000
258     231500
1201    197900
Name: SalePrice, dtype: int64

In [17]:
print(ytest.shape)
ytest.head()

(292,)


620     67000
305    305900
849    187000
962    155000
940    150900
Name: SalePrice, dtype: int64

**Question 6** (25 points) <br/>
**a)** Now create your linear regression model, using `LinearRegression().fit` on your training data. Assign it to `regmodel`. <br/>
**b)** Calculate the predictions the model gives for your test dataset, assigning it to the variable `predictions`. <br/>
**c)** Print the $RMSE$ and $R^2$ values (for both the training and testing data). <br/>
**d)** Copy your `train_test_split` command from Question 5 to the top of the next cell, so that you can run that cell and do one complete random sampling of the train/test data, linear model generation, model statistics, and model equation. 

*Note:* The code to print the regression equation for you is included. 

In [41]:
# Create the linear regression model
Xtrain, Xtest, ytrain, ytest = train_test_split(df, answers['SalePrice'], test_size=1/5)
regmodel = LinearRegression().fit(Xtrain, ytrain)

# Calculate the predictions the model gives
predictions_test = regmodel.predict(Xtest)
predictions_train = regmodel.predict(Xtrain)

#Print the RMSE and R^2 values for the testing set
r2_test = regmodel.score(Xtest, ytest)
rmse_test = math.sqrt(mean_squared_error(ytest, predictions_test))
print('The RMSE is ' + str(rmse_test) + '. The R^2' + ' is ' + str(r2_test) + ' for the testing set.') 

#Print the RMSE and R^2 values for the training set
r2_train = regmodel.score(Xtrain, ytrain)
rmse_train = math.sqrt(mean_squared_error(ytrain, predictions_train))
print('The RMSE is ' + str(rmse_train) + '. The R^2' + ' is ' + str(r2_train) + ' for the training set.') 

# Prints the regression equation
print('SalePrice = ')
for i in np.arange(regmodel.coef_.size):
    print('({:.3f})*{} + '.format(regmodel.coef_[i],df.columns[i]))

print('({:.3f})'.format(regmodel.intercept_))

The RMSE is 31127.136243915185. The R^2 is 0.8458811275533938 for the testing set.
The RMSE is 31498.38600470383. The R^2 is 0.842802593232179 for the training set.
SalePrice = 
(0.447)*LotArea + 
(13403.108)*OverallQual + 
(6324.997)*OverallCond + 
(388.256)*YearBuilt + 
(12.194)*TotalBsmtSF + 
(46.761)*GrLivArea + 
(15182.454)*BsmtFullBath + 
(13620.970)*BsmtHalfBath + 
(7369.390)*FullBath + 
(314.964)*HalfBath + 
(-5526.799)*BedroomAbvGr + 
(9827.781)*GarageCars + 
(-11714.927)*MSZoning_C (all) + 
(11033.402)*MSZoning_FV + 
(1097.927)*MSZoning_RH + 
(684.545)*MSZoning_RL + 
(-1100.947)*MSZoning_RM + 
(5047.196)*Neighborhood_Blmngtn + 
(-3178.460)*Neighborhood_Blueste + 
(7087.001)*Neighborhood_BrDale + 
(-7289.734)*Neighborhood_BrkSide + 
(3338.040)*Neighborhood_ClearCr + 
(-13873.250)*Neighborhood_CollgCr + 
(14255.966)*Neighborhood_Crawfor + 
(-25850.348)*Neighborhood_Edwards + 
(-19673.848)*Neighborhood_Gilbert + 
(-14052.201)*Neighborhood_IDOTRR + 
(9513.645)*Neighborhood_Meadow

**Question 7** (25 points) <br/>
Run the `train_test_split` and `LinearRegression().fit` several times, noting the RMSE and R^2 train and test results, along with the regression equation found by the model. Answer the following questions.

What appears to be the rough average for RMSE? <br/>
What appears to be the rough averages for R^2? <br/>

Name three things that make sense in your model.<br/>
Name two things that don't make sense in your model.<br/>

Based on your model, what would you expect the three wealthiest neighborhoods to be? <br/>
What would you expect the three poorest neighborhoods to be? 

Are you concerned you overfit your model? 

Put your answers here.

*One final note:* If we had more than just ~1500 homes, we could create machine learning models using all of the factors available in `fulldf`. 

In [44]:
# What appears to be the rough average for RMSE?
print('\r\nThe average for RMSE is 32,196.74 from runing train_test_split and LinearRegression().fit a total of 10 times including both the training and testing sets.')

# What appears to be the rough averages for R^2?
print('\r\nThe average for R^2 is 0.8288 from runing train_test_split and LinearRegression().fit a total of 10 times including both the training and testing sets.')

# Name three things that make sense in your model.
print('\r\nThree things that make sense in your model:')
print('1. Typically, the neighborhood has a big effect on the Sale Price with a strong positive or strong negative correlation.')
print('2. Garage cars has a high positive correlation with the Sale Price.')
print('3. Overall Quality has a high positve correlation with the Sale Price.')

# Name two things that don't make sense in your model.
print('\r\nTwo things that do not make sense in your model:')
print('1. Sale Conditon Normal has a slightly negative effect on the Sale Price. You would expect a normal sale condition to have a positive correlation with Sale Price.')
print('2. Lot area does has a relatively neutral corrrelation with the Sale Price. You would assume the lot size would have a positive correlation with Sale Price because\r\nlarger lots should have a higher Sale Price than smaller lots.')

# Based on your model, what would you expect the three wealthiest neighborhoods to be?

# Limit the masked_array and masked_df to only the Neighborhoods
import numpy.ma as ma
masked_array = regmodel.coef_[17:42]
masked_array = ma.array(masked_array, mask=False)
masked_df = df[['Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber', 'Neighborhood_Veenker']]

# Use the maximum index to get the wealthiest neighborhood name.
# Mask the wealthiest neighborhood name in order to get the next wealthiest neighborhood
max_first = masked_array.argmax()
max_first_neighborhood = masked_df.columns[max_first]
masked_array.mask[max_first] = True

max_second = masked_array.argmax()
max_second_neighborhood = masked_df.columns[max_second]
masked_array.mask[max_second] = True

max_third = masked_array.argmax()
max_third_neighborhood = masked_df.columns[max_third]
masked_array.mask[max_third] = True

print('\r\nThe three wealthiest neighborhoods are #1 ' + str(max_first_neighborhood) + ', #2 ' + str(max_second_neighborhood) + ', #3 ' + str(max_third_neighborhood) + '.')

# What would you expect the three poorest neighborhoods to be?
# Use the minimum index to get the poorest neighborhood name.
# Mask the poorest neighborhood name in order to get the next poorest neighborhood
min_first = masked_array.argmin()
min_first_neighborhood = masked_df.columns[min_first]
masked_array.mask[min_first] = True

min_second = masked_array.argmin()
min_second_neighborhood = masked_df.columns[min_second]
masked_array.mask[min_second] = True

min_third = masked_array.argmin()
min_third_neighborhood = masked_df.columns[min_third]
masked_array.mask[min_third] = True

print('The three poorest neighborhoods are #1 ' + str(min_first_neighborhood) + ', #2 ' + str(min_second_neighborhood) + ', #3 ' + str(min_third_neighborhood) + '.\r\nWhen the R^2 values are both 0.84 (the best fit of the data), the poorest neighborhoods are #1 Neighborhood_Edwards, #2 Neighborhood_OldTown, #3 Neighborhood_NWAmes.')
print('Note: The poorest neighborhoods change depending on the model coefficients / the fit of the model.')

#Are you concerned you overfit your model?
print('\r\nThe R^2 values between the training and testing sets are usually similar, where the training R^2 value is ' + str(r2_train) + ' and the testing R^2 value is ' + str(r2_test) + '.\r\nWhen the R^2 values are similar, the model is not overfit. On occasion, the R^2 will differ by about 0.35 between the training and testing sets; in this case, the model is overfit and should be rerun.')




The average for RMSE is 32,196.74 from runing train_test_split and LinearRegression().fit a total of 10 times including both the training and testing sets.

The average for R^2 is 0.8288 from runing train_test_split and LinearRegression().fit a total of 10 times including both the training and testing sets.

Three things that make sense in your model:
1. Typically, the neighborhood has a big effect on the Sale Price with a strong positive or strong negative correlation.
2. Garage cars has a high positive correlation with the Sale Price.
3. Overall Quality has a high positve correlation with the Sale Price.

Two things that do not make sense in your model:
1. Sale Conditon Normal has a slightly negative effect on the Sale Price. You would expect a normal sale condition to have a positive correlation with Sale Price.
2. Lot area does has a relatively neutral corrrelation with the Sale Price. You would assume the lot size would have a positive correlation with Sale Price because
larger l